
# Scalable Growth Infrastructure & Automation (Portfolio Project)
**Goal:** Demonstrate an end-to-end analytics workflow that integrates multiple data sources, applies data quality checks, builds KPI-ready datasets, and exports clean tables for BI dashboards (Power BI / Excel).

**Tech:** Python (Pandas), SQL-ready outputs, BI-ready CSV exports  
**Data:** Synthetic dataset (safe for GitHub) placed in `data/` folder



##  Setup & Folder Structure

Make sure your repository (or local folder) looks like this:

```
project-root/
  data/
    customer_data.csv
    sales_data.csv
    performance_metrics_daily.csv
    sales_leaderboard_monthly.csv
    data_quality_issues_sample.csv
  notebooks/
    growth_pipeline.ipynb   (this notebook)
```

If your files are somewhere else, update `DATA_DIR` below.


In [ ]:

from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA_DIR = Path("data")  # <-- change if needed
OUT_DIR = Path("outputs")
OUT_DIR.mkdir(exist_ok=True)

print("DATA_DIR:", DATA_DIR.resolve())
print("OUT_DIR:", OUT_DIR.resolve())


## 1) Load Data

In [ ]:

customers = pd.read_csv(DATA_DIR / "customer_data.csv")
sales = pd.read_csv(DATA_DIR / "sales_data.csv")
daily_kpis = pd.read_csv(DATA_DIR / "performance_metrics_daily.csv")
leaderboard = pd.read_csv(DATA_DIR / "sales_leaderboard_monthly.csv")
dq_issues = pd.read_csv(DATA_DIR / "data_quality_issues_sample.csv")

display(customers.head())
display(sales.head())
print("customers:", customers.shape)
print("sales:", sales.shape)
print("daily_kpis:", daily_kpis.shape)
print("leaderboard:", leaderboard.shape)
print("dq_issues:", dq_issues.shape)


## 2) Quick Profiling

In [ ]:

def profile_df(df, name):
    print(f"\n=== {name} ===")
    print(df.dtypes)
    print("Missing values (top 10):")
    display(df.isna().sum().sort_values(ascending=False).head(10))
    print("Duplicate rows:", df.duplicated().sum())

profile_df(customers, "customers")
profile_df(sales, "sales")
profile_df(daily_kpis, "daily_kpis")
profile_df(leaderboard, "leaderboard")



## 3) Data Quality Checks (QA/QC)

We run common checks:
- Missing values
- Duplicates
- Join integrity (sales must map to a customer)
- Outlier review (revenue)

We do not delete suspicious rows silently — we **flag** and **document**.


In [ ]:

# 3.1 Type fixes
sales["date"] = pd.to_datetime(sales["date"], errors="coerce")
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")

# 3.2 Join integrity
missing_customers = sales.loc[~sales["customer_id"].isin(customers["customer_id"])]
print("Sales rows with missing customer_id match:", len(missing_customers))

# 3.3 Duplicate transactions
dup_tx = sales[sales.duplicated(subset=["transaction_id"], keep=False)]
print("Duplicate transaction_id rows:", len(dup_tx))

# 3.4 Revenue outliers (simple IQR rule)
q1 = sales["revenue"].quantile(0.25)
q3 = sales["revenue"].quantile(0.75)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr

outliers = sales[(sales["revenue"] < lower) | (sales["revenue"] > upper)]
print("Revenue outliers flagged (IQR):", len(outliers))

# Save QC findings
qc_summary = pd.DataFrame({
    "check": ["missing_customer_match", "duplicate_transaction_id", "revenue_outliers_iqr"],
    "flagged_rows": [len(missing_customers), len(dup_tx), len(outliers)]
})
qc_summary.to_csv(OUT_DIR / "qc_summary.csv", index=False)
qc_summary


## 4) System Integration: Build a Unified Dataset

In [ ]:

# Merge sales + customer master data
df = sales.merge(customers, on="customer_id", how="left")

# Basic cleaning / standardization
df["revenue"] = pd.to_numeric(df["revenue"], errors="coerce")
df["approved"] = df["approved"].astype(int)
df["processing_failure"] = df["processing_failure"].astype(int)
df["disputed"] = df["disputed"].astype(int)

# Add time features
df["year"] = df["date"].dt.year
df["month"] = df["date"].dt.to_period("M").astype(str)
df["day_of_week"] = df["date"].dt.day_name()

display(df.head())
print("Unified dataset shape:", df.shape)

df.to_csv(OUT_DIR / "unified_sales_customer.csv", index=False)



## 5) KPI Tables (SQL-ready Outputs)

We create:
- Daily KPI table
- Monthly KPI table (by region/channel/product)
- Leaderboard rollups (monthly)

These outputs can be loaded into Power BI or used to build SQL views.


In [ ]:

# 5.1 Daily KPIs (recomputed from unified data)
daily = df.groupby(df["date"].dt.date).agg(
    total_transactions=("transaction_id", "count"),
    total_revenue=("revenue", "sum"),
    approval_rate=("approved", "mean"),
    processing_failure_rate=("processing_failure", "mean"),
    dispute_rate=("disputed", "mean"),
).reset_index().rename(columns={"date":"date"})

daily["total_revenue"] = daily["total_revenue"].round(2)
daily["approval_rate"] = daily["approval_rate"].round(4)
daily["processing_failure_rate"] = daily["processing_failure_rate"].round(4)
daily["dispute_rate"] = daily["dispute_rate"].round(4)
daily["revenue_7d_ma"] = daily["total_revenue"].rolling(7).mean().round(2)
daily["approval_7d_ma"] = daily["approval_rate"].rolling(7).mean().round(4)
daily["month"] = pd.to_datetime(daily["date"]).dt.to_period("M").astype(str)

# 5.2 Monthly KPIs segmented (region/channel/product)
monthly = df.groupby(["month", "region", "channel", "product_tier"]).agg(
    total_transactions=("transaction_id", "count"),
    total_revenue=("revenue", "sum"),
    approval_rate=("approved", "mean"),
    dispute_rate=("disputed", "mean"),
).reset_index()

monthly["total_revenue"] = monthly["total_revenue"].round(2)
monthly["approval_rate"] = monthly["approval_rate"].round(4)
monthly["dispute_rate"] = monthly["dispute_rate"].round(4)

# Save outputs
daily.to_csv(OUT_DIR / "kpi_daily.csv", index=False)
monthly.to_csv(OUT_DIR / "kpi_monthly_segmented.csv", index=False)
leaderboard.to_csv(OUT_DIR / "leaderboard_monthly.csv", index=False)

display(daily.head())
display(monthly.head())
print("Saved KPI outputs to:", OUT_DIR.resolve())


## 6) Simple Visualizations (Trend & Monitoring)

In [ ]:

# Revenue trend
plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(daily["date"]), daily["total_revenue"])
plt.title("Total Revenue (Daily)")
plt.xlabel("Date")
plt.ylabel("Revenue")
plt.show()

# Approval rate trend
plt.figure(figsize=(10,4))
plt.plot(pd.to_datetime(daily["date"]), daily["approval_rate"])
plt.title("Approval Rate (Daily)")
plt.xlabel("Date")
plt.ylabel("Approval Rate")
plt.ylim(0, 1)
plt.show()

# Monthly revenue by region
monthly_region = df.groupby(["month", "region"]).agg(total_revenue=("revenue","sum")).reset_index()
pivot = monthly_region.pivot(index="month", columns="region", values="total_revenue").fillna(0)

plt.figure(figsize=(12,5))
plt.plot(pivot.index, pivot.values)
plt.title("Monthly Revenue by Region")
plt.xlabel("Month")
plt.ylabel("Revenue")
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



## 7) Optional: Example SQL Views (copy/paste)

If you load the exported CSVs into a database, these are example views you can create.

> Note: SQL syntax may vary slightly by database (PostgreSQL, SQL Server, Snowflake).


In [ ]:

-- Example: Daily KPI view
-- CREATE VIEW vw_kpi_daily AS
-- SELECT
--   date,
--   total_transactions,
--   total_revenue,
--   approval_rate,
--   processing_failure_rate,
--   dispute_rate,
--   revenue_7d_ma,
--   approval_7d_ma
-- FROM kpi_daily;

-- Example: Monthly segmented KPI view
-- CREATE VIEW vw_kpi_monthly_segmented AS
-- SELECT
--   month,
--   region,
--   channel,
--   product_tier,
--   total_transactions,
--   total_revenue,
--   approval_rate,
--   dispute_rate
-- FROM kpi_monthly_segmented;



## 8) What to Upload to GitHub

Upload these folders/files:
- `data/` (CSV files) — optional if you want the data public
- `notebooks/` (this notebook)
- `outputs/` (generated KPI tables)
- `README.md` (project explanation)

If you keep the data private, include instructions in the README for how to generate/download the data.
